-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Auto Load Data to Multiplex Bronze

The chief architect has decided that rather than connecting directly to Kafka, a source system will send raw records as JSON files to cloud object storage. In this notebook, you'll build a multiplex table that will ingest these records with Auto Loader and store the entire history of this incremental feed. The initial table will store data from all of our topics and have the following schema. 

| Field | Type |
| --- | --- |
| key | BINARY |
| value | BINARY |
| topic | STRING |
| partition | LONG |
| offset | LONG
| timestamp | LONG |
| date | DATE |
| week_part | STRING |

This single table will drive the majority of the data through the target architecture, feeding three interdependent data pipelines.

<img src="https://files.training.databricks.com/images/ade/ADE_arch_bronze.png" width="60%" />

**NOTE**: Details on additional configurations for connecting to Kafka are available <a href="https://docs.databricks.com/spark/latest/structured-streaming/kafka.html" target="_blank">here</a>.


## Learning Objectives

By the end of this lesson, you should be able to:
- Describe a multiplex design
- Apply Auto Loader to incrementally process records
- Configure trigger intervals
- Use "trigger-available-now" logic to execute triggered incremental loading of data.

The following cell declares the paths needed throughout this notebook.

In [0]:
%run ../Includes/Classroom-Setup-3.1

Creating the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_3_1"

Creating date_lookup...(3 seconds / 1,096 records)

Loading batch #1 to the daily stream...(8 seconds, 417,276 records)

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/3.1
 DA.paths.user_db: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/3.1/3_1.db
 DA.paths.checkpoints: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/3.1/_checkpoints
 DA.paths.source_daily: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/3.1/streams/daily

Predefined tables in dbacademy_odl_user_771624_databrickslabs_com_adewd_3_1:
 date_lookup

Validating datasets...(1 seconds)

Setup completed in 13 seconds

<img src="https://files.training.databricks.com/images/icon_warn_24.png"> All records are being stored on the DBFS root for this training example.

Setting up separate databases and storage accounts for different layers
of data is preferred in both development and production environments.

## Examine Source Data

Data files are being written to the path specified by the variable below.

Run the following cell to examine the schema in the source data and determine if anything needs to be changed as it's being ingested.

In [0]:
spark.read.json(DA.paths.source_daily).printSchema()

root
-- key: string (nullable = true)
-- offset: long (nullable = true)
-- partition: long (nullable = true)
-- timestamp: long (nullable = true)
-- topic: string (nullable = true)
-- value: string (nullable = true)

## Prepare Data to Join with Date Lookup Table
The initialization script has loaded a **`date_lookup`** table. This table has a number of pre-computed date values. Note that additional fields indicating holidays or financial quarters might often be added to this table for later data enrichment.

Pre-computing and storing these values is especially important based on our desire to partition our data by year and week, using the string pattern **`YYYY-WW`**. While Spark has both **`year`** and **`weekofyear`** functions built in, the **`weekofyear`** function may not provide expected behavior for dates falling in the last week of December or <a href="https://spark.apache.org/docs/2.3.0/api/sql/#weekofyear" target="_blank">first week of January</a>, as it defines week 1 as the first week with >3 days.

While this edge case is esoteric to Spark, a **`date_lookup`** table that will be used across the organization is important for making sure that data is consistently enriched with date-related details.

In [0]:
%sql

DESCRIBE date_lookup

col_name,data_type,comment
date,date,
week,int,
year,int,
month,int,
dayofweek,int,
dayofmonth,int,
dayofyear,int,
week_part,string,
,,
# Partitioning,,


The current table being implemented requires that we capture the accurate **`week_part`** for each **`date`**.

The following call creates the **`DataFrame`** needed for the subsequent join operation.

In [0]:
date_lookup_df = spark.table("date_lookup").select("date", "week_part")

In [0]:
display(date_lookup_df)

date,week_part
2020-02-16,2020-7
2020-02-17,2020-8
2020-02-18,2020-8
2020-02-19,2020-8
2020-02-20,2020-8
2020-02-21,2020-8
2020-02-22,2020-8
2020-02-23,2020-8
2020-02-24,2020-9
2020-02-25,2020-9


Working with the JSON data stored in the **`DA.paths.source_daily`** location, transform the **`timestamp`** column as necessary to join it with the **`date`** column.

In [0]:
# TODO
from pyspark.sql import functions as F
json_df = spark.read.json(DA.paths.source_daily)
 
joined_df = (json_df.join(F.broadcast(date_lookup_df),
                          F.to_date((F.col("timestamp")/1000).cast("timestamp")) == F.col("date"),  # Insert the matching condition
                          "left"))
 
display(joined_df)

key,offset,partition,timestamp,topic,value,date,week_part
MTE5NzE1,65087128,0,1575158410818,bpm,eyJkZXZpY2VfaWQiOiAxMTk3MTUsICJ0aW1lIjogMTU3NTE1ODQwMCwgImhlYXJ0cmF0ZSI6IDY2LjYyMjQ0OTczMTU3OTc2fQ==,2019-12-01,2019-48
MTA3OTUy,65102257,2,1575158411394,bpm,eyJkZXZpY2VfaWQiOiAxMDc5NTIsICJ0aW1lIjogMTU3NTE1ODQwMSwgImhlYXJ0cmF0ZSI6IDExNi4wMzEyNTg5ODAxMjA4N30=,2019-12-01,2019-48
MTYxOTEw,65087131,0,1575158417678,bpm,eyJkZXZpY2VfaWQiOiAxNjE5MTAsICJ0aW1lIjogMTU3NTE1ODQwMywgImhlYXJ0cmF0ZSI6IDgyLjMzMzg1OTEyMDg3MzE3fQ==,2019-12-01,2019-48
MTE2NzA4,64899138,3,1575158421868,bpm,eyJkZXZpY2VfaWQiOiAxMTY3MDgsICJ0aW1lIjogMTU3NTE1ODQxMiwgImhlYXJ0cmF0ZSI6IDgzLjEzNDUyNzA2MDg3NzA5fQ==,2019-12-01,2019-48
MTYxMzQz,64899148,3,1575158430841,bpm,eyJkZXZpY2VfaWQiOiAxNjEzNDMsICJ0aW1lIjogMTU3NTE1ODQyMCwgImhlYXJ0cmF0ZSI6IDU5LjcxODMyNzY1MTYyODA0fQ==,2019-12-01,2019-48
MTEzMDI0,65002565,4,1575158432217,bpm,eyJkZXZpY2VfaWQiOiAxMTMwMjQsICJ0aW1lIjogMTU3NTE1ODQyMywgImhlYXJ0cmF0ZSI6IDk0LjcyNTkyNjkxMDg1MjAxfQ==,2019-12-01,2019-48
MTQxNjg3,65102289,2,1575158437927,bpm,eyJkZXZpY2VfaWQiOiAxNDE2ODcsICJ0aW1lIjogMTU3NTE1ODQzMCwgImhlYXJ0cmF0ZSI6IDg0LjM4NzM2MzY4NjEzMDU0fQ==,2019-12-01,2019-48
MTQyODM4,64899162,3,1575158445876,bpm,eyJkZXZpY2VfaWQiOiAxNDI4MzgsICJ0aW1lIjogMTU3NTE1ODQzNCwgImhlYXJ0cmF0ZSI6IDg1Ljc2MzkzNjgwMDQ1MTAzfQ==,2019-12-01,2019-48
MTk3OTMw,65087190,0,1575158460427,bpm,eyJkZXZpY2VfaWQiOiAxOTc5MzAsICJ0aW1lIjogMTU3NTE1ODQ1MCwgImhlYXJ0cmF0ZSI6IDg0LjYyMDk2MzIyMTczNTA3fQ==,2019-12-01,2019-48
MTEwMzY3,65102354,2,1575158482354,bpm,eyJkZXZpY2VfaWQiOiAxMTAzNjcsICJ0aW1lIjogMTU3NTE1ODQ3MSwgImhlYXJ0cmF0ZSI6IDc2LjUwOTk5NTg3NjA2Njc0fQ==,2019-12-01,2019-48


## Define Triggered Incremental Auto Loading to Multiplex Bronze Table

Below is starter code for a function to incrementally process data from the source directory to the bronze table, creating the table during the initial write.

Fill in the missing code to:
- Configure the stream to use Auto Loader
- Configure Auto Loader to use the JSON format
- Perform a broadcast join with the date_lookup table
- Partition the data by the **`topic`** and **`week_part`** fields

In [0]:
# TODO
def process_bronze():
    query = (spark.readStream
                  .format("cloudFiles")
                  .option("cloudFiles.format", "json")
                  .option("cloudFiles.schemaLocation", f"{DA.paths.checkpoints}/bronze_schema")
                  .load(DA.paths.source_daily)
                  .join(F.broadcast(date_lookup_df), F.to_date((F.col("timestamp")/1000).cast("timestamp")) == F.col("date"), "left")
                  .writeStream
                  .option("checkpointLocation", f"{DA.paths.checkpoints}/bronze")
                  .partitionBy("topic", "week_part")
                  .trigger(availableNow=True)
                  .table("bronze"))
 
    query.awaitTermination()

Run the cell below to process an incremental batch of data.

In [0]:
process_bronze()

Review the count of processed records.

In [0]:
%sql
SELECT COUNT(*) FROM bronze

count(1)
417276


Preview the data to ensure records are being ingested correctly.

In [0]:
%sql
SELECT * FROM bronze

key,offset,partition,timestamp,topic,value,_rescued_data,date,week_part
MTE5NzE1,65087128,0,1575158410818,bpm,eyJkZXZpY2VfaWQiOiAxMTk3MTUsICJ0aW1lIjogMTU3NTE1ODQwMCwgImhlYXJ0cmF0ZSI6IDY2LjYyMjQ0OTczMTU3OTc2fQ==,null,2019-12-01,2019-48
MTA3OTUy,65102257,2,1575158411394,bpm,eyJkZXZpY2VfaWQiOiAxMDc5NTIsICJ0aW1lIjogMTU3NTE1ODQwMSwgImhlYXJ0cmF0ZSI6IDExNi4wMzEyNTg5ODAxMjA4N30=,null,2019-12-01,2019-48
MTYxOTEw,65087131,0,1575158417678,bpm,eyJkZXZpY2VfaWQiOiAxNjE5MTAsICJ0aW1lIjogMTU3NTE1ODQwMywgImhlYXJ0cmF0ZSI6IDgyLjMzMzg1OTEyMDg3MzE3fQ==,null,2019-12-01,2019-48
MTE2NzA4,64899138,3,1575158421868,bpm,eyJkZXZpY2VfaWQiOiAxMTY3MDgsICJ0aW1lIjogMTU3NTE1ODQxMiwgImhlYXJ0cmF0ZSI6IDgzLjEzNDUyNzA2MDg3NzA5fQ==,null,2019-12-01,2019-48
MTYxMzQz,64899148,3,1575158430841,bpm,eyJkZXZpY2VfaWQiOiAxNjEzNDMsICJ0aW1lIjogMTU3NTE1ODQyMCwgImhlYXJ0cmF0ZSI6IDU5LjcxODMyNzY1MTYyODA0fQ==,null,2019-12-01,2019-48
MTEzMDI0,65002565,4,1575158432217,bpm,eyJkZXZpY2VfaWQiOiAxMTMwMjQsICJ0aW1lIjogMTU3NTE1ODQyMywgImhlYXJ0cmF0ZSI6IDk0LjcyNTkyNjkxMDg1MjAxfQ==,null,2019-12-01,2019-48
MTQxNjg3,65102289,2,1575158437927,bpm,eyJkZXZpY2VfaWQiOiAxNDE2ODcsICJ0aW1lIjogMTU3NTE1ODQzMCwgImhlYXJ0cmF0ZSI6IDg0LjM4NzM2MzY4NjEzMDU0fQ==,null,2019-12-01,2019-48
MTQyODM4,64899162,3,1575158445876,bpm,eyJkZXZpY2VfaWQiOiAxNDI4MzgsICJ0aW1lIjogMTU3NTE1ODQzNCwgImhlYXJ0cmF0ZSI6IDg1Ljc2MzkzNjgwMDQ1MTAzfQ==,null,2019-12-01,2019-48
MTk3OTMw,65087190,0,1575158460427,bpm,eyJkZXZpY2VfaWQiOiAxOTc5MzAsICJ0aW1lIjogMTU3NTE1ODQ1MCwgImhlYXJ0cmF0ZSI6IDg0LjYyMDk2MzIyMTczNTA3fQ==,null,2019-12-01,2019-48
MTEwMzY3,65102354,2,1575158482354,bpm,eyJkZXZpY2VfaWQiOiAxMTAzNjcsICJ0aW1lIjogMTU3NTE1ODQ3MSwgImhlYXJ0cmF0ZSI6IDc2LjUwOTk5NTg3NjA2Njc0fQ==,null,2019-12-01,2019-48


The **`DA.daily_stream.load()`** code below is a helper class to land new data in the source directory.

Executing the following cell should successfully process a new batch.

In [0]:
DA.daily_stream.load()

Loading batch #2 to the daily stream...(7 seconds, 462,415 records)

In [0]:
process_bronze()

Confirm the count is now higher.

In [0]:
%sql
SELECT COUNT(*) FROM bronze

count(1)
879691


Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()

Dropping the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_3_1"
Removing the working directory "dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/3.1"
Validating datasets...(1 seconds)

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>